In [2]:
import pandas as pd 


In [3]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

In [4]:
recipes_df = pd.read_parquet("recipes.parquet")
reviews_df = pd.read_parquet("reviews.parquet")

# Display

In [5]:
recipes_df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."


In [6]:
reviews_df.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25 21:44:00+00:00,2000-01-25 21:44:00+00:00
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17 16:49:59+00:00,2001-10-17 16:49:59+00:00
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25 09:00:00+00:00,2000-02-25 09:00:00+00:00
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13 21:15:00+00:00,2000-03-13 21:15:00+00:00
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28 12:51:00+00:00,2000-03-28 12:51:00+00:00


# Pre-processing

In [7]:
nutritional_cols = ["RecipeId", "Name", "Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent", "RecipeServings"]
nutritional_df = recipes_df[nutritional_cols]

In [8]:
nutritional_df.head()

,RecipeId,Name,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
0,38.0,Low-Fat Berry Blue Frozen Dessert,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0
1,39.0,Biryani,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0
2,40.0,Best Lemonade,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0
3,41.0,Carina's Tofu-Vegetable Kebabs,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0
4,42.0,Cabbage Soup,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0


In [9]:
nutritional_df = nutritional_df.dropna(axis=0)

Divide all columns in nutritional_df by the by RecipeServings

In [10]:
cols_to_divide = ["Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent"]
nutritional_df[cols_to_divide] = nutritional_df[cols_to_divide].div(nutritional_df["RecipeServings"], axis=0)
nutritional_df.drop("RecipeServings", axis=1)

,RecipeId,Name,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,38.0,Low-Fat Berry Blue Frozen Dessert,42.725000,0.625000,0.325000,2.000000,7.450000,9.275000,0.900000,7.550000,0.800000
1,39.0,Biryani,185.116667,9.800000,2.766667,62.133333,61.400000,14.066667,1.500000,3.400000,10.566667
2,40.0,Best Lemonade,77.775000,0.050000,0.000000,0.000000,0.450000,20.375000,0.100000,19.300000,0.075000
3,41.0,Carina's Tofu-Vegetable Kebabs,268.050000,12.000000,1.900000,0.000000,779.300000,32.100000,8.650000,16.050000,14.650000
4,42.0,Cabbage Soup,25.900000,0.100000,0.025000,0.000000,239.825000,6.275000,1.200000,4.425000,1.075000
...,...,...,...,...,...,...,...,...,...,...,...
522509,541376.0,Spanish Coffee with Tia Maria,84.300000,2.100000,1.200000,6.800000,15.700000,16.600000,0.400000,15.400000,0.600000
522510,541377.0,Slow-Cooker Classic Coffee Cake,29.908333,1.650000,0.875000,8.591667,26.950000,3.458333,0.066667,2.066667,0.400000
522512,541379.0,Meg's Fresh Ginger Gingerbread,39.575000,1.562500,0.950000,6.800000,34.775000,6.062500,0.100000,2.850000,0.487500
522513,541380.0,Roast Prime Rib au Poivre with Mixed Peppercorns,257.925000,21.550000,8.925000,54.225000,95.787500,0.400000,0.087500,0.012500,14.625000


In [31]:
nutritional_df[cols_to_divide].iloc

In [32]:
def recipe_name_to_id(name):
    return nutritional_df[nutritional_df['Name'] == name]['RecipeId']

In [56]:
nutritional_df[nutritional_df['RecipeId'] == 54]

,RecipeId,Name,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
16,54.0,Carrot Cake,0.0001,0.001061,0.000425,0.000178,0.000063,0.000052,0.000076,0.000044,0.00021,12.0


In [57]:
original_nutritional_df = nutritional_df
nutritional_df_copy = nutritional_df.copy()
cols_to_divide = cols_to_divide

# normalize the attributes
scaler = MinMaxScaler()
knn = NearestNeighbors(metric='euclidean')
nutritional_df_copy[cols_to_divide] = scaler.fit_transform(nutritional_df_copy[cols_to_divide])
knn.fit(nutritional_df_copy[cols_to_divide])

input_recipe = nutritional_df_copy.loc[nutritional_df_copy["RecipeId"] == 54, cols_to_divide]
distances, indices = knn.kneighbors(input_recipe, n_neighbors=11)  # +1 to exclude the recipe itself

closest_indices = indices[0][1:]  # Exclude the first element (recipe itself)
original_nutritional_df.iloc[closest_indices]


,RecipeId,Name,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
472229,489638.0,Storm Warning Glazed Cinnamon Coffee Cake,0.000103,0.001073,0.000425,0.000189,0.000039,0.000054,0.000057,0.000049,0.000227,12.0
443278,459624.0,Strawberry Cake,0.000112,0.001065,0.000434,0.000161,0.000042,0.000063,0.000062,0.000058,0.000202,12.0
72397,77150.0,Traditional Black Russian Bundt Cake,0.000098,0.001049,0.000382,0.000158,0.000058,0.000044,0.000062,0.000037,0.000206,12.0
182508,190783.0,Ho Ho Ho Rum Cake,0.000102,0.001006,0.000416,0.000182,0.000059,0.000051,0.000057,0.000045,0.000202,12.0
218293,227581.0,Cranberry Apple Cake,0.000105,0.001069,0.000442,0.000199,0.000039,0.000055,0.000124,0.000041,0.000236,12.0
61564,65963.0,Lemon Key Lime Cake,0.000096,0.001053,0.000425,0.000195,0.000050,0.000048,0.000024,0.000036,0.000172,12.0
401614,416338.0,Irish Cream Almond Cake,0.000103,0.001088,0.000468,0.000204,0.000059,0.000050,0.000062,0.000044,0.000248,12.0
18730,22110.0,Sock-It-To-Me Cake,0.000088,0.001030,0.000477,0.000204,0.000037,0.000040,0.000076,0.000032,0.000231,12.0
403283,418057.0,Carrot Cake With Baby Food Carrots,0.000118,0.001038,0.000477,0.000212,0.000048,0.000069,0.000062,0.000066,0.000236,12.0
401653,416380.0,Pantry Cake,0.000104,0.000994,0.000399,0.000201,0.000039,0.000058,0.000067,0.000053,0.000227,12.0


In [47]:
recommender =  RecipeRecommender(nutritional_df,cols_to_divide)

In [52]:
def recipe_name_to_id(name):
    return nutritional_df[nutritional_df['Name'] == name]['RecipeId']

In [54]:
nutritional_df[nutritional_df['Name'] == "Green Olives"]['RecipeId']

2304    3826.0
Name: RecipeId, dtype: float64

,RecipeId,Name,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
2304,3826.0,Green Olives,19657.5,2128.5,277.9,0.0,349356.4,192.8,165.0,27.8,51.5,1.0
185088,193435.0,Tennessee Moonshine,434360.2,840.8,118.0,0.0,31777.7,108294.6,1748.6,90682.3,1980.8,1.0
46472,50478.0,Creole Seasoning,473.3,16.2,3.1,0.0,288247.2,96.1,39.3,15.1,19.8,1.0
369043,382448.0,Homemade Laundry and Dishwasher Detergent,0.0,0.0,0.0,0.0,250148.3,0.0,0.0,0.0,0.0,1.0
420547,436028.0,Pickled Jalapeno Peppers,801.5,8.7,0.9,0.0,226502.5,127.0,40.2,81.3,19.7,1.0
